In [ ]:
%cd /data/cmpe258-sp24/miaosen/CMPE249/HW_1/working

/fs/atipa/data/cmpe258-sp24/miaosen/CMPE249/HW_1/working


In [ ]:
!ls

final_model_2.pth  final_model.pth  Kitti


In [ ]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)

2.4.1
0.19.1


In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

False
0


/data/cmpe258-sp24/miaosen/miniconda3/envs/CMPE249py311/lib/python3.11/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /opt/conda/conda-bld/pytorch_1724789145202/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/data/cmpe258-sp24/miaosen/miniconda3/envs/CMPE249py311/lib/python3.11/site-packages/torch/cuda/__init__.py:654: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
#creating the symlink for detection
!git clone https://github.com/lkk688/DeepDataMiningLearning.git
%cd DeepDataMiningLearning
!pip install flit
!flit install --symlink

In [ ]:
from DeepDataMiningLearning.detection.dataset_kitti import MyKittiDetection

In [ ]:
import DeepDataMiningLearning.detection.transforms as T
def get_transformsimple(train):
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.ToDtype(torch.float, scale=True))
    # if train:
    #     transforms.append(RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [ ]:
rootPath='/data/cmpe258-sp24/miaosen/CMPE249/HW_1/working/Kitti'
is_train = True
kittidataset = MyKittiDetection(rootPath, train=True, transform=get_transformsimple(is_train))

In [ ]:
print(kittidataset.INSTANCE_CATEGORY_NAMES)

['Car', 'Van', 'Truck', 'Pedestrian', 'Person_sitting', 'Cyclist', 'Tram', 'Misc', 'DontCare']


In [ ]:
print("Dataset len:", len(kittidataset))

Dataset len: 7481


In [ ]:
img, target = kittidataset[0]

In [ ]:
from DeepDataMiningLearning.detection import utils

In [ ]:
BATCH_SIZE=16
WORKERS = 16
train_collate_fn = utils.collate_fn
train_sampler = torch.utils.data.RandomSampler(kittidataset)
train_batch_sampler = torch.utils.data.BatchSampler(train_sampler, BATCH_SIZE, drop_last=True)

In [ ]:
data_loader = torch.utils.data.DataLoader(
        kittidataset, batch_sampler=train_batch_sampler, num_workers=WORKERS, collate_fn=train_collate_fn
    )

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained pre-trained
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 10  # 1 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# move model to the right device
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
save_path = '/data/cmpe258-sp24/miaosen/CMPE249/HW_1/working/final_model_2.pth'

In [ ]:
from tqdm import tqdm

In [ ]:
# Define optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# Learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Number of epochs
num_epochs = 50
i = 0

for epoch in tqdm(range(num_epochs), desc="Processing"):
    model.train()
    for images, targets in data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    print(f"Epoch: {epoch+1}/{num_epochs}, Loss: {losses.item()}")
    lr_scheduler.step()
    if i % 5 == 0:
        torch.save(model.state_dict(), save_path)
        print(f"Model saved to '{save_path}'")
    i = i+1

Processing:   0%|                                                                                                         | 0/50 [00:00<?, ?it/s]

Epoch: 1/50, Loss: 0.42612579464912415


Processing:   2%|█▊                                                                                           | 1/50 [12:38<10:19:38, 758.74s/it]

Model saved to '/data/cmpe258-sp24/miaosen/CMPE249/HW_1/working/final_model_2.pth'
Epoch: 2/50, Loss: 0.5486985445022583


Processing:   4%|███▋                                                                                         | 2/50 [25:04<10:00:42, 750.89s/it]

...

Model saved to '/data/cmpe258-sp24/miaosen/CMPE249/HW_1/working/final_model_2.pth'
Epoch: 28/50, Loss: 0.42429405450820923


Processing:  56%|██████████████████████████████████████████████████▉                                        | 28/50 [5:49:45<4:35:01, 750.05s/it]

Model saved to '/data/cmpe258-sp24/miaosen/CMPE249/HW_1/working/final_model_2.pth'
Epoch: 29/50, Loss: 0.4108785092830658


Processing:  58%|████████████████████████████████████████████████████▊                                      | 29/50 [6:02:11<4:22:04, 748.80s/it]

Model saved to '/data/cmpe258-sp24/miaosen/CMPE249/HW_1/working/final_model_2.pth'
